In [1]:
%pip install sly

Note: you may need to restart the kernel to use updated packages.


In [4]:
from sly import Lexer
from sly import Parser


In [91]:
class CalcLexer(Lexer):
    # Set of token names. This is always required
    tokens = {
        'ID', 'EQUALS', 'LPAREN', 'RPAREN', 'COMMA', 'PLUS', 'DOT','LBRACKET', 'RBRACKET', 'TERNARY'
    }
    
    # String containing ignored characters between tokens
    ignore = ' \t'

    # Regular expression rules for tokens
    ID      = r'[a-zA-Z_][a-zA-Z0-9_]*'
    EQUALS  = r'='
    LPAREN  = r'\('
    RPAREN  = r'\)'
    COMMA   = r','
    PLUS    = r'\+'
    DOT     = r'\.'
    LBRACKET = r'\['
    RBRACKET = r'\]'
    TERNARY  = r'\?'

In [74]:
class Node:
    def __init__(self, type_, children=None, value=None):
        self.type = type_
        self.children = children
        self.value = value


In [133]:

from sly import Parser

class CalcParser(Parser):
    tokens = CalcLexer.tokens

    precedence = (
        ('left', 'PLUS'),
    )

    def __init__(self):
        self.names = {}

    
    
    @_('expression_list')
    def start(self, p):
        return p.expression_list

    @_('expression')
    def expression_list(self, p):
        return Node('ExpressionList', children=[p.expression])

    @_('expression_list PLUS expression')
    def expression_list(self, p):
        p.expression_list.children.append(p.expression)
        return p.expression_list

    @_('parallel_expr')
    def expression(self, p):
        return p.parallel_expr


    @_('command')
    def expression(self, p):
        return p.command

    @_('LPAREN expression_list RPAREN')
    def parallel_expr(self, p):
        return Node('ParallelBlock', value=p.expression_list)
    
    @_('ID LPAREN param_list RPAREN')
    def command(self, p):
        return Node('Command', value=[p.ID, p.param_list])

    @_('ID LPAREN RPAREN')
    def command(self, p):
        return Node('Command', value=[p.ID, []])

    @_('param')
    def param_list(self, p):
        return Node('ParamList',children=[p.param])

    @_('param_list COMMA param')
    def param_list(self, p):
        p.param_list.children.append(p.param)
        return p.param_list

    @_('ID EQUALS ID')
    def param(self, p):
        return Node('Param', value=[p.ID0, Node('ParamValue', value=[p.ID1])])

    @_('ID EQUALS param_value')
    def param(self, p):
        return Node('Param', value=[p.ID, Node('ParamValue', value=p.param_value)])

    @_('ID DOT ID')
    def param_value(self, p):
        return Node('ParamValue', value=[p.ID0,p.ID1])

def print_tree(node, indent=0):
    if not node:
        return
    print('  ' * indent + str(node.type))
    if node.children:
        for child in node.children:
            print_tree(child, indent + 1)
    elif node.value is not None:
        print('  ' * (indent + 1) + str(node.value))
        if isinstance(node.value, Node):
            print_tree(node.value, indent + 1)


if __name__ == '__main__':
    lexer = CalcLexer()
    parser = CalcParser()
    data = '(c1(p1=i1)) + c2(p2=i2,p8=o1.m1) + (c3(p3=i4) + c4(p3=i6))'
    result = parser.parse(lexer.tokenize(data))
    print_tree(result)




ExpressionList
  ParallelBlock
    ExpressionList
      Command
        ['c1', <__main__.Node object at 0x1211cd210>]
  Command
    ['c2', <__main__.Node object at 0x1211cd1d0>]
  ParallelBlock
    ExpressionList
      Command
        ['c3', <__main__.Node object at 0x1211ce950>]
      Command
        ['c4', <__main__.Node object at 0x1211cec10>]
